In [1]:
import os, json
import pandas as pd

# this finds our json files
path_to_json = '../CORD/train/json/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

# here I define my pandas Dataframe with the columns I want to get from the json
jsons_data = pd.DataFrame(columns=['validline'])

f = open("receipt_words.csv", "w")
print("filename",",","index",",","x1",",",'x2',",",'x3',",",'x4',",",'y1',",",'y2',",",'y3',",",'y4',",",'is_key',",",'row_id',",",'text',",",'category',",","group_id",file=f)


# we need both the json and an index number so use enumerate()
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        json_text = json.load(json_file)
        
       # print(json_text)

        # here you need to know the layout of your json and each json has to have
        # the same structure (obviously not the structure I have here)
        lines = json_text['valid_line']
        
        for x in lines:
            words = x['words']
            category = x['category']
            group_id = x['group_id']
            for y in words:
                print(js,index,y['quad']['x1'],y['quad']['x2'],y['quad']['x3'],y['quad']['x4'],y['quad']['y1'],y['quad']['y2'],y['quad']['y3'],y['quad']['y4'],y['is_key'],y['row_id'],"\""+y['text']+"\"",category,group_id,file=f,sep=",")

       # city = json_text['features'][0]['properties']['name']
        #lonlat = json_text['features'][0]['geometry']['coordinates']
        # here I push a list of data into a pandas DataFrame at row given by 'index'
        #f.close()
        #jsons_data.loc[index] = [words, city, lonlat]

# now that we have the pertinent json data in our DataFrame let's look at it
f.close()

In [2]:
import pandas as pd
receipts_df = pd.read_csv("receipt_words.csv")
receipts_df.columns = receipts_df.columns.str.strip()
receipts_df['text'] = receipts_df['text'].str.strip()


In [3]:
receipts_df.dtypes

filename    object
index        int64
x1           int64
x2           int64
x3           int64
x4           int64
y1           int64
y2           int64
y3           int64
y4           int64
is_key       int64
row_id       int64
text        object
category    object
group_id     int64
dtype: object

In [4]:
char2idx = {"START":0,"END":1,"a":2,"b":3,"c":4,"d":5,"e":6,"f":7,"g":8,"h":9,"i":10,"j":11,"k":12,"l":13,"m":14,"n":15,"o":16,"p":17,"q":18,"r":19,"s":20,"t":21,"u":22,"v":23,"w":24,"x":25,"y":26,"z":27,"0":28,"1":29,"2":30,"3":31,"4":32,"5":33,"6":34,"7":35,"8":36,"9":37,".":38,"-":39,",":40,":":41," ":42}

In [5]:
#p_test.SentimentText=p_test.SentimentText.astype(str)
receipts_df.text = receipts_df.text.astype(str)
receipts_df.dtypes

filename    object
index        int64
x1           int64
x2           int64
x3           int64
x4           int64
y1           int64
y2           int64
y3           int64
y4           int64
is_key       int64
row_id       int64
text        object
category    object
group_id     int64
dtype: object

In [6]:
words = receipts_df['text']

In [7]:
indexed_words = []
j=43
for word in words:
    indexed_word = []
    #print(word)
    word = word.lower();
    for i in range(len(word)):
        token = word[i]
        if token not in char2idx:
            char2idx[token] = j
            j += 1
        indexed_word.append(char2idx[token])
    indexed_words.append(indexed_word)

receipts_df["indexed_words"] = indexed_words;

In [8]:
import numpy as np
def get_bigram_probs(words, V, start_idx, end_idx, smoothing=1):
  # structure of bigram probability matrix will be:
  # (last word, current word) --> probability
  # we will use add-1 smoothing
  # note: we'll always ignore this from the END token
  bigram_probs = np.ones((V, V)) * smoothing
  for word in words:
    for i in range(len(word)):
      
      if i == 0:
        # beginning word
        word[i]
        bigram_probs[start_idx, word[i]] += 1
      else:
        # middle word
        bigram_probs[word[i-1], word[i]] += 1

      # if we're at the final word
      # we update the bigram for last -> current
      # AND current -> END token
      if i == len(word) - 1:
        # final word
        bigram_probs[word[i], end_idx] += 1

  # normalize the counts along the rows to get probabilities
  bigram_probs /= bigram_probs.sum(axis=1, keepdims=True)
  return bigram_probs

In [9]:
start_idx = char2idx['START']
end_idx = char2idx['END']
V = len(char2idx)



In [10]:
bigram_probs = get_bigram_probs(indexed_words, V, start_idx, end_idx, smoothing=0.1)

In [11]:
bigram_probs

array([[5.27303790e-06, 5.27303790e-06, 1.57716564e-02, ...,
        1.10733796e-04, 5.80034169e-05, 1.10733796e-04],
       [1.56250000e-02, 1.56250000e-02, 1.56250000e-02, ...,
        1.56250000e-02, 1.56250000e-02, 1.56250000e-02],
       [1.57918009e-05, 7.97643863e-02, 8.05381846e-04, ...,
        1.57918009e-05, 1.57918009e-05, 1.57918009e-05],
       ...,
       [1.19047619e-02, 2.50000000e-01, 1.19047619e-02, ...,
        1.19047619e-02, 1.19047619e-02, 1.19047619e-02],
       [1.19047619e-02, 1.19047619e-02, 1.19047619e-02, ...,
        1.19047619e-02, 1.30952381e-01, 1.19047619e-02],
       [1.19047619e-02, 1.19047619e-02, 1.30952381e-01, ...,
        1.19047619e-02, 1.19047619e-02, 1.19047619e-02]])

In [12]:
 # a function to calculate normalized log prob score
  # for a sentence
def get_score(word):
    #print("word")
    #print(word)
    score = 0
    if len(word) > 0 :
        for i in range(len(word)):
          if i == 0:
            # beginning word
            score += np.log(bigram_probs[start_idx, word[i]])
          else:
            # middle word
            score += np.log(bigram_probs[word[i-1], word[i]])
        # final word
        score += np.log(bigram_probs[word[i-1], end_idx])

    # normalize the score
    # return score / (len(word) + 1)
    if score == 0:
        return score;
    else :
        return  ((-1)*(len(word) + 1)/score)*100

In [13]:
from future.utils import iteritems

idx2char = dict((v, k) for k, v in iteritems(char2idx))
def get_word(word):
    return ''.join(idx2char[i] for i in word)

In [14]:
receipts_df['text_score'] = receipts_df.apply(lambda row : get_score(row['indexed_words']), axis = 1)

In [15]:
receipts_df

,filename,index,x1,x2,x3,x4,y1,y2,y3,y4,is_key,row_id,text,category,group_id,indexed_words,text_score
0,receipt_00580.json,0,250,325,325,250,833,834,869,868,0,879802,Tebu,menu.nm,3,"[21, 6, 3, 22]",36.669749
1,receipt_00580.json,0,340,434,433,339,832,833,868,867,0,879802,Lemon,menu.nm,3,"[13, 6, 14, 16, 15]",35.199925
2,receipt_00580.json,0,217,227,228,217,835,834,869,869,0,879802,1,menu.cnt,3,[29],75.625303
3,receipt_00580.json,0,841,954,954,841,824,824,864,864,0,879802,22.000,menu.price,3,"[30, 30, 38, 28, 28, 28]",66.930682
4,receipt_00580.json,0,249,342,344,249,921,920,990,992,1,879803,Total,total.total_price,4,"[21, 16, 21, 2, 13]",62.899046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18953,receipt_00281.json,786,186,449,451,188,671,654,691,707,1,1623131,SUBTOTAL,sub_total.subtotal_price,10,"[20, 22, 3, 21, 16, 21, 2, 13]",61.298959
18954,receipt_00281.json,786,630,712,712,630,642,642,678,678,0,1623131,"9,500",sub_total.subtotal_price,10,"[37, 40, 33, 28, 28]",54.478680
18955,receipt_00281.json,786,188,242,242,188,714,714,750,750,1,1623132,TAX,sub_total.tax_price,10,"[21, 2, 25]",40.665969
18956,receipt_00281.json,786,258,310,310,258,708,708,744,744,1,1623132,10%,sub_total.tax_price,10,"[29, 28, 43]",43.846155


In [16]:
categories = receipts_df['category'] 
category2idx = {}
idx2category = {}
j=0;
for category in categories:
    if category not in category2idx:
        category2idx[category] = j
        idx2category[j] = category
        j+=1

category2idx

def get_index_for_category(category):
    return category2idx[category]


def get_category_for_index(index):
    return idx2category[index]
        

In [17]:
receipts_df['category_idx'] = receipts_df.apply(lambda row : get_index_for_category(row['category']), axis = 1)

In [18]:
data = receipts_df[['x1','x2','x3','x4','y1','y2','y3','y4','is_key','text_score','category_idx','index']]



import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
#x = X.values #returns a numpy array
x = data.values #returns a numpy array
#min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler = preprocessing.StandardScaler()
x_scaled = min_max_scaler.fit_transform(x)
X = pd.DataFrame(x_scaled)
X

,0,1,2,3,4,5,6,7,8,9,10,11
0,-0.456113,-0.461640,-0.461996,-0.456009,-0.094436,-0.092283,-0.093020,-0.095220,-0.571625,-0.758032,-0.882977,-1.703774
1,-0.179346,-0.162279,-0.165442,-0.182323,-0.096199,-0.094046,-0.094737,-0.096936,-0.571625,-0.852567,-0.882977,-1.703774
2,-0.557594,-0.730791,-0.728346,-0.557489,-0.090911,-0.092283,-0.093020,-0.093504,-0.571625,1.747492,-0.689413,-1.703774
3,1.361326,1.265869,1.265160,1.361392,-0.110302,-0.109913,-0.101603,-0.102085,-0.571625,1.188275,-0.495848,-1.703774
4,-0.459188,-0.414951,-0.409825,-0.459085,0.060692,0.059334,0.114682,0.117589,1.749399,0.928971,-0.302283,-1.703774
...,...,...,...,...,...,...,...,...,...,...,...,...
18953,-0.652925,-0.121082,-0.116016,-0.646667,-0.380014,-0.409620,-0.398564,-0.371529,1.749399,0.826057,0.471976,1.734247
18954,0.712460,0.601231,0.600658,0.712540,-0.431136,-0.430776,-0.420880,-0.421299,-0.571625,0.387394,0.471976,1.734247
18955,-0.646775,-0.689595,-0.689904,-0.646667,-0.304213,-0.303841,-0.297289,-0.297732,1.749399,-0.501005,1.052671,1.734247
18956,-0.431512,-0.502837,-0.503184,-0.431408,-0.314790,-0.314419,-0.307588,-0.308030,1.749399,-0.296463,1.052671,1.734247


In [19]:


from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=4, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors(X)
distances
from sklearn.metrics.pairwise import cosine_similarity

print(cosine_similarity(X.loc[[0]],X.loc[[1]])[0,0])
print(cosine_similarity(X.loc[[0]],X.loc[[2]]))
print(cosine_similarity(X.loc[[0]],X.loc[[768]]))
from scipy.spatial import distance
d = distance.euclidean(X.loc[[0]], X.loc[[1]])
print("Euclidean distance: ",d)
d = distance.euclidean(X.loc[[0]], X.loc[[2]])
print("Euclidean distance: ",d)
d = distance.euclidean(X.loc[[0]], X.loc[[768]])
print("Euclidean distance: ",d)

def getcosine(X,Y):
    return cosine_similarity(X,Y)[0,0];

def getneighourscore(index,X):
    return X[9].tolist()[0];

#X["n1_cd"] = X.apply(lambda row : getcosine(X.loc[[row.name]],X.loc[[indices[row.name,1]]]), axis = 1)
#X["n2_cd"] = X.apply(lambda row : getcosine(X.loc[[row.name]],X.loc[[indices[row.name,2]]]), axis = 1)
#X["n3_cd"] = X.apply(lambda row : getcosine(X.loc[[row.name]],X.loc[[indices[row.name,3]]]), axis = 1)

#X["n1_ed"] = X.apply(lambda row : distance.euclidean(X.loc[[row.name]],X.loc[[indices[row.name,1]]]), axis = 1)
#X["n2_ed"] = X.apply(lambda row : distance.euclidean(X.loc[[row.name]],X.loc[[indices[row.name,2]]]), axis = 1)
#X["n3_ed"] = X.apply(lambda row : distance.euclidean(X.loc[[row.name]],X.loc[[indices[row.name,3]]]), axis = 1)

X["n1_score"] = X.apply(lambda row : getneighourscore(row.name,X.loc[[indices[row.name,1]]]), axis = 1)
X["n2_score"] = X.apply(lambda row : getneighourscore(row.name,X.loc[[indices[row.name,2]]]), axis = 1)
X["n3_score"] = X.apply(lambda row : getneighourscore(row.name,X.loc[[indices[row.name,3]]]), axis = 1)


X

0.9688203682864794
[[0.54823113]]
[[0.92378828]]
Euclidean distance:  0.5813910736052611
Euclidean distance:  2.545409923306853
Euclidean distance:  1.01013185083155


,0,1,2,3,4,5,6,7,8,9,10,11,n1_score,n2_score,n3_score
0,-0.456113,-0.461640,-0.461996,-0.456009,-0.094436,-0.092283,-0.093020,-0.095220,-0.571625,-0.758032,-0.882977,-1.703774,-0.830488,-0.535017,-0.698732
1,-0.179346,-0.162279,-0.165442,-0.182323,-0.096199,-0.094046,-0.094737,-0.096936,-0.571625,-0.852567,-0.882977,-1.703774,-0.837338,-1.047618,-0.721962
2,-0.557594,-0.730791,-0.728346,-0.557489,-0.090911,-0.092283,-0.093020,-0.093504,-0.571625,1.747492,-0.689413,-1.703774,1.747492,1.747492,1.747492
3,1.361326,1.265869,1.265160,1.361392,-0.110302,-0.109913,-0.101603,-0.102085,-0.571625,1.188275,-0.495848,-1.703774,1.310614,1.234175,1.188275
4,-0.459188,-0.414951,-0.409825,-0.459085,0.060692,0.059334,0.114682,0.117589,1.749399,0.928971,-0.302283,-1.703774,0.928971,0.928971,0.928971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18953,-0.652925,-0.121082,-0.116016,-0.646667,-0.380014,-0.409620,-0.398564,-0.371529,1.749399,0.826057,0.471976,1.734247,0.826057,0.826057,0.826057
18954,0.712460,0.601231,0.600658,0.712540,-0.431136,-0.430776,-0.420880,-0.421299,-0.571625,0.387394,0.471976,1.734247,0.448033,0.477326,-0.230154
18955,-0.646775,-0.689595,-0.689904,-0.646667,-0.304213,-0.303841,-0.297289,-0.297732,1.749399,-0.501005,1.052671,1.734247,-0.461989,-0.296463,-0.789256
18956,-0.431512,-0.502837,-0.503184,-0.431408,-0.314790,-0.314419,-0.307588,-0.308030,1.749399,-0.296463,1.052671,1.734247,-0.330923,-0.296463,-0.501005


In [20]:
X

,0,1,2,3,4,5,6,7,8,9,10,11,n1_score,n2_score,n3_score
0,-0.456113,-0.461640,-0.461996,-0.456009,-0.094436,-0.092283,-0.093020,-0.095220,-0.571625,-0.758032,-0.882977,-1.703774,-0.830488,-0.535017,-0.698732
1,-0.179346,-0.162279,-0.165442,-0.182323,-0.096199,-0.094046,-0.094737,-0.096936,-0.571625,-0.852567,-0.882977,-1.703774,-0.837338,-1.047618,-0.721962
2,-0.557594,-0.730791,-0.728346,-0.557489,-0.090911,-0.092283,-0.093020,-0.093504,-0.571625,1.747492,-0.689413,-1.703774,1.747492,1.747492,1.747492
3,1.361326,1.265869,1.265160,1.361392,-0.110302,-0.109913,-0.101603,-0.102085,-0.571625,1.188275,-0.495848,-1.703774,1.310614,1.234175,1.188275
4,-0.459188,-0.414951,-0.409825,-0.459085,0.060692,0.059334,0.114682,0.117589,1.749399,0.928971,-0.302283,-1.703774,0.928971,0.928971,0.928971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18953,-0.652925,-0.121082,-0.116016,-0.646667,-0.380014,-0.409620,-0.398564,-0.371529,1.749399,0.826057,0.471976,1.734247,0.826057,0.826057,0.826057
18954,0.712460,0.601231,0.600658,0.712540,-0.431136,-0.430776,-0.420880,-0.421299,-0.571625,0.387394,0.471976,1.734247,0.448033,0.477326,-0.230154
18955,-0.646775,-0.689595,-0.689904,-0.646667,-0.304213,-0.303841,-0.297289,-0.297732,1.749399,-0.501005,1.052671,1.734247,-0.461989,-0.296463,-0.789256
18956,-0.431512,-0.502837,-0.503184,-0.431408,-0.314790,-0.314419,-0.307588,-0.308030,1.749399,-0.296463,1.052671,1.734247,-0.330923,-0.296463,-0.501005


In [21]:
X.dtypes

0           float64
1           float64
2           float64
3           float64
4           float64
5           float64
6           float64
7           float64
8           float64
9           float64
10          float64
11          float64
n1_score    float64
n2_score    float64
n3_score    float64
dtype: object

In [22]:
X.loc[[2]]

,0,1,2,3,4,5,6,7,8,9,10,11,n1_score,n2_score,n3_score
2,-0.557594,-0.730791,-0.728346,-0.557489,-0.090911,-0.092283,-0.09302,-0.093504,-0.571625,1.747492,-0.689413,-1.703774,1.747492,1.747492,1.747492


In [23]:
X.loc[[768]]

,0,1,2,3,4,5,6,7,8,9,10,11,n1_score,n2_score,n3_score
768,-0.274677,-0.425937,-0.4263,-0.280727,-0.418796,-0.423724,-0.422596,-0.421299,-0.571625,-1.466584,-0.882977,-1.555055,-1.466584,-1.346576,-1.301942


In [24]:
import tensorflow as tf

from tensorflow.keras import models
from tensorflow.keras import layers

In [25]:

#data = receipts_df[['x1','x2','x3','x4','y1','y2','y3','y4','is_key','text_score','category_idx','index']]
data = X

data['category_idx']=receipts_df['category_idx'];

#from sklearn.neighbors import NearestNeighbors
#nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X)
#distances, indices = nbrs.kneighbors(data[0])

dnnModel = models.Sequential()
# Layer 1 = input layer
# specify the input size in the first layer.

dnnModel.add(layers.Dense(60, activation='elu', input_shape= (15,)))

# Layer 2 = hidden layer 
#dnnModel.add(layers.Dense(60, activation='relu'))

# Layer 3 = hidden layer 
#dnnModel.add(layers.Dense(30, activation='linear'))

# Layer 4 = output layer
dnnModel.add(layers.Dense(len(category2idx), activation='sigmoid'))

dnnModel.summary()
dnnModel.compile( optimizer = 'adam', loss = 'categorical_hinge', metrics=['accuracy'] )
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)
y = data.pop('category_idx')
X = data

import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
x = X.values #returns a numpy array
#min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler = preprocessing.StandardScaler()
x_scaled = min_max_scaler.fit_transform(x)
X = pd.DataFrame(x_scaled)

X_train,X_test,y_train,y_test = train_test_split(X.index,y.index,test_size=0.2)
xtrain=X.iloc[X_train]
xtest=X.iloc[X_test]
y_train=y.iloc[y_train]
y_test=y.iloc[y_test]

from imblearn.over_sampling import RandomOverSampler
from collections import Counter
print('Before Resampled dataset shape' )
print( y_train.shape)
sm = RandomOverSampler('all',random_state=42)
xtrain, y_train = sm.fit_resample(xtrain, y_train)
print(' Resampled dataset shape' )
print( y_train.shape)

ytrain = np.zeros((len(y_train), len(idx2category.values())))
ytest = np.zeros((len(y_test), len(idx2category.values())))
for i in range(len(y_train)):
    #print(y_train.values[i])
    ytrain[i][y_train[i]]=1
    
for i in range(len(y_test)):
    #print(y_train.values[i])
    ytrain[i][y_test.values[i]]=1
    


xtrain=tf.convert_to_tensor(xtrain)

ytraintens=tf.convert_to_tensor(ytrain)

h  = dnnModel.fit( xtrain, ytraintens, epochs=10, batch_size=32,validation_split=0.2)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 60)                960       
_________________________________________________________________
dense_1 (Dense)              (None, 30)                1830      
Total params: 2,790
Trainable params: 2,790
Non-trainable params: 0
_________________________________________________________________


Using TensorFlow backend.


Before Resampled dataset shape
(15166,)
 Resampled dataset shape
(117488,)
Train on 93990 samples, validate on 23498 samples
Epoch 1/10
93990/93990 [==============================] - 9s 92us/sample - loss: 0.9571 - accuracy: 0.4789 - val_loss: 1.2317 - val_accuracy: 0.0786
Epoch 2/10
93990/93990 [==============================] - 8s 81us/sample - loss: 0.8938 - accuracy: 0.5309 - val_loss: 1.4675 - val_accuracy: 0.0908
Epoch 3/10
93990/93990 [==============================] - 8s 81us/sample - loss: 0.8192 - accuracy: 0.5152 - val_loss: 1.5662 - val_accuracy: 0.1000
Epoch 4/10
93990/93990 [==============================] - 7s 79us/sample - loss: 0.7523 - accuracy: 0.4588 - val_loss: 1.5704 - val_accuracy: 0.1075
Epoch 5/10
93990/93990 [==============================] - 8s 87us/sample - loss: 0.7047 - accuracy: 0.4731 - val_loss: 1.5088 - val_accuracy: 0.1075
Epoch 6/10
93990/93990 [==============================] - 8s 90us/sample - loss: 0.6715 - accuracy: 0.4947 - val_loss: 1.4887 - va

In [26]:
import seaborn as sns
from matplotlib import pyplot as plt
# visualise training history
plt.plot(h.history ['accuracy'])
plt.plot(h.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc="lower right")
plt.show()

<Figure size 640x480 with 1 Axes>